In [1]:
import glob,os
import openai
import pickle
from dotenv import load_dotenv

In [2]:
allims = glob.glob(os.path.join("/Users/damoncrockett/samplebooks/src/assets/img/_modelrank/","*.jpg"))

In [3]:
basepaths = [os.path.basename(item) for item in allims]

In [4]:
import sys
sys.path.append("/Users/damoncrockett/ivpy/src")
from ivpy import *

In [5]:
AWS = "https://samplebook.photos/img/"

In [6]:
imurls = [AWS+item for item in basepaths]

# API

In [7]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [8]:
client = openai.OpenAI()

In [9]:
with open("/Users/damoncrockett/samplebooks/src/assets/txt/modelrank_prompt.txt", 'r') as f:
    prompt = f.read()

In [10]:
def cleanup(s):
    return s.replace("\n","")

In [11]:
def get_and_clean_response(url):
    
    response = client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {
          "role": "user",
          "content": [
            {"type": "text", "text": prompt},
            {
              "type": "image_url",
              "image_url": {
                "url": url,
              },
            },
          ],
        }
      ],
      max_tokens=1024,
    )
    
    return cleanup(response.choices[0].message.content)

In [12]:
responses = []
for url in imurls:
    r = get_and_clean_response(url)
    responses.append(r)

In [13]:
all_responses = {}

for i,url in enumerate(imurls):
    all_responses[url] = {"gpt-4o-2024-08-06":responses[i]}

In [15]:
import pickle

with open("responses_openai.pkl", "wb") as f:
    pickle.dump(all_responses, f)